In [62]:
import pandas as pd
from sqlalchemy import create_engine

# Create a database and tables out of 'game_sales.csv' and 'game_reviews.csv' 

engine = create_engine('sqlite:///game_sales_db.db')

game_sales_df = pd.read_csv('game_sales.csv', encoding='ISO-8859-1')
game_sales_df.to_sql('game_sales', engine, if_exists='replace', index=False)

game_reviews_df = pd.read_csv('game_reviews.csv', encoding='ISO-8859-1')
game_reviews_df.to_sql('reviews', engine, if_exists='replace', index=False)

400

In [63]:
# Examine the 'game_sales' table.

with engine.connect() as con:
    game_sales = pd.read_sql_query(""" 
    SELECT *
    FROM game_sales
    LIMIT 10; """, con)

game_sales

,Name,Platform,Publisher,Developer,Total_Shipped,Year
0,7 Days to Die for PC,PC,The Fun Pimps,The Fun Pimps,4.18,2013
1,ARK: Survival Evolved for PC,PC,Studio Wildcard,Studio Wildcard,4.50,2015
2,Age of Empires II: HD Edition for PC,PC,Microsoft Studios,Hidden Path Entertainment,5.82,2013
3,Animal Crossing: City Folk for Wii,Wii,Nintendo,Nintendo EAD,4.32,2008
4,Animal Crossing: New Horizons for NS,NS,Nintendo,Nintendo,13.41,2020
5,Animal Crossing: New Leaf for 3DS,3DS,Nintendo,Nintendo EAD,12.55,2013
6,Animal Crossing: Wild World for DS,DS,Nintendo,Nintendo EAD,11.75,2005
7,Arma 2: Operation Arrowhead for PC,PC,Meridian4,Bohemia Interactive,4.51,2010
8,Arma III for PC,PC,Bohemia Interactive,Bohemia Interactive,4.00,2013
9,Assassin's Creed II for PS3,PS3,Ubisoft,Ubisoft Montreal,5.57,2009


In [64]:
# Examine the 'reviews' table.

with engine.connect() as con:
    game_reviews = pd.read_sql_query(""" 
    SELECT *
    FROM reviews
    LIMIT 10; """, con)

game_reviews

,Name,Critic_Score,User_Score
0,Wii Sports for Wii,7.7,8.0
1,Super Mario Bros. for NES,10.0,8.2
2,Counter-Strike: Global Offensive for PC,8.0,7.5
3,Mario Kart Wii for Wii,8.2,9.1
4,PLAYERUNKNOWN'S BATTLEGROUNDS for PC,8.6,4.7
5,Minecraft for PC,10.0,7.8
6,Wii Sports Resort for Wii,8.0,8.8
7,Pokemon Red / Green / Blue Version for GB,9.4,8.8
8,New Super Mario Bros. for DS,9.1,8.1
9,New Super Mario Bros. Wii for Wii,8.6,9.2


In [65]:
# Let's find the ten best-selling video games in game_sales_data.

with engine.connect() as con:
    top_10_sold = pd.read_sql_query(""" 
    SELECT *
    FROM game_sales
    ORDER BY total_shipped DESC
    LIMIT 10; """, con)

top_10_sold

,Name,Platform,Publisher,Developer,Total_Shipped,Year
0,Wii Sports for Wii,Wii,Nintendo,Nintendo EAD,82.90,2006
1,Super Mario Bros. for NES,NES,Nintendo,Nintendo EAD,40.24,1985
2,Counter-Strike: Global Offensive for PC,PC,Valve,Valve Corporation,40.00,2012
3,Mario Kart Wii for Wii,Wii,Nintendo,Nintendo EAD,37.32,2008
4,PLAYERUNKNOWN'S BATTLEGROUNDS for PC,PC,PUBG Corporation,PUBG Corporation,36.60,2017
5,Minecraft for PC,PC,Mojang,Mojang AB,33.15,2010
6,Wii Sports Resort for Wii,Wii,Nintendo,Nintendo EAD,33.13,2009
7,Pokemon Red / Green / Blue Version for GB,GB,Nintendo,Game Freak,31.38,1998
8,New Super Mario Bros. for DS,DS,Nintendo,Nintendo EAD,30.80,2006
9,New Super Mario Bros. Wii for Wii,Wii,Nintendo,Nintendo EAD,30.30,2009


In [66]:
# Let's determine how many games in the game_sales table are missing both a user_score and a critic_score.

with engine.connect() as con:
    missing_user_critic_score = pd.read_sql_query(""" 
    SELECT COUNT(game_sales.name) AS missing_user_critic_reviews
    FROM game_sales
    LEFT JOIN reviews
    USING (name)
    WHERE reviews.critic_score IS NULL AND reviews.user_score IS NULL; """, con)

missing_user_critic_score

,missing_user_critic_reviews
0,31


In [67]:
# Let's find the years with the highest average critic_score.

with engine.connect() as con:
    highest_avg_critic_score = pd.read_sql_query(""" 
    SELECT game_sales.year, ROUND(AVG(reviews.critic_score), 2) AS avg_critic_score
    FROM game_sales
    INNER JOIN reviews
    USING (name)
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 10; """, con)

highest_avg_critic_score

,Year,avg_critic_score
0,1990,9.80
1,1992,9.67
2,1998,9.32
3,2020,9.20
4,1993,9.10
5,1995,9.07
6,2004,9.03
7,1982,9.00
8,2002,8.99
9,1999,8.93


In [68]:
# Let's find game critics' ten favorite years, with more than four games released in a year order to be considered.

with engine.connect() as con:
    ten_critic_fav_years = pd.read_sql_query(""" 
    SELECT game_sales.year, ROUND(AVG(reviews.critic_score), 2) AS avg_critic_score, COUNT(game_sales.name) AS num_games
    FROM game_sales
    INNER JOIN reviews
    USING (name)
    GROUP BY 1
    HAVING COUNT(game_sales.name)>4
    ORDER BY 2 DESC
    LIMIT 10; """, con)

ten_critic_fav_years

,Year,avg_critic_score,num_games
0,1998,9.32,10
1,2004,9.03,11
2,2002,8.99,9
3,1999,8.93,11
4,2001,8.82,13
5,2011,8.76,26
6,2016,8.67,13
7,2013,8.66,18
8,2008,8.63,20
9,2017,8.62,13


In [69]:
# Let's identify years in the first critics' favorite list but not in the second due to having fewer than four reviewed games using SQL and sort by critic scores. 

with engine.connect() as con:
    except_years = pd.read_sql_query(""" 
    SELECT year, avg_critic_score 
    FROM (
        SELECT game_sales.year, ROUND(AVG(reviews.critic_score), 2) AS avg_critic_score
        FROM game_sales
        INNER JOIN reviews
        USING (name)
        GROUP BY 1
        ORDER BY 2 DESC
        LIMIT 10)
    EXCEPT 
    SELECT year, avg_critic_score
    FROM (
        SELECT game_sales.year, ROUND(AVG(reviews.critic_score), 2) AS avg_critic_score, COUNT(game_sales.name) AS num_games
        FROM game_sales
        INNER JOIN reviews
        USING (name)
        GROUP BY 1
        HAVING COUNT(game_sales.name)>4
        ORDER BY 2 DESC
        LIMIT 10) 
    ORDER BY avg_critic_score DESC;""", con)

except_years

,year,avg_critic_score
0,1990,9.80
1,1992,9.67
2,2020,9.20
3,1993,9.10
4,1995,9.07
5,1982,9.00


In [70]:
# Let's find game users' ten favorite years, with more than four games released in a year order to be considered.

with engine.connect() as con:
    ten_user_fav_years = pd.read_sql_query(""" 
    SELECT game_sales.year, ROUND(AVG(reviews.user_score), 2) AS avg_user_score, COUNT(game_sales.name) AS num_games
    FROM game_sales
    INNER JOIN reviews
    USING (name)
    GROUP BY 1
    HAVING COUNT(game_sales.name)>4
    ORDER BY 2 DESC
    LIMIT 10; """, con)

ten_user_fav_years

,Year,avg_user_score,num_games
0,1997,9.50,8
1,1998,9.40,10
2,2010,9.24,23
3,2009,9.18,20
4,2008,9.03,20
5,1996,9.00,5
6,2006,8.95,16
7,2005,8.95,13
8,2002,8.80,9
9,2000,8.80,8


In [71]:
# Let's create a list of games with years that both players and critics loved

with engine.connect() as con:
    intersect_years = pd.read_sql_query(""" 
    SELECT year
    FROM (
        SELECT game_sales.year, ROUND(AVG(reviews.user_score), 2) AS avg_user_score, COUNT(game_sales.name) AS num_games
        FROM game_sales
        INNER JOIN reviews
        USING (name)
        GROUP BY 1
        HAVING COUNT(game_sales.name)>4
        ORDER BY 2 DESC
        LIMIT 10)
    INTERSECT 
    SELECT year
    FROM (
        SELECT game_sales.year, ROUND(AVG(reviews.critic_score), 2) AS avg_critic_score, COUNT(game_sales.name) AS num_games
        FROM game_sales
        INNER JOIN reviews
        USING (name)
        GROUP BY 1
        HAVING COUNT(game_sales.name)>4
        ORDER BY 2 DESC
        LIMIT 10);""", con)

intersect_years

,year
0,1998
1,2002
2,2008


In [72]:
# Let's add a column 'total_games_sold' to the previos task.

with engine.connect() as con:
    intersect_years_total_sold = pd.read_sql_query(""" 
    SELECT year, SUM(total_shipped) AS total_games_sold
    FROM game_sales
    WHERE year IN (
    SELECT year
    FROM (
        SELECT game_sales.year, ROUND(AVG(reviews.user_score), 2) AS avg_user_score, COUNT(game_sales.name) AS num_games
        FROM game_sales
        INNER JOIN reviews
        USING (name)
        GROUP BY 1
        HAVING COUNT(game_sales.name)>4
        ORDER BY 2 DESC
        LIMIT 10)
    INTERSECT 
    SELECT year
    FROM (
        SELECT game_sales.year, ROUND(AVG(reviews.critic_score), 2) AS avg_critic_score, COUNT(game_sales.name) AS num_games
        FROM game_sales
        INNER JOIN reviews
        USING (name)
        GROUP BY 1
        HAVING COUNT(game_sales.name)>4
        ORDER BY 2 DESC
        LIMIT 10)
    )
    GROUP BY 1
    ORDER BY 2 DESC;""", con)

intersect_years_total_sold

,Year,total_games_sold
0,2008,175.07
1,1998,101.52
2,2002,58.67
